In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
 #from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # e.g., "gpt-4o", "gpt-4o-mini", "omni-gpt-4", "omni-gpt-4-mini"
    # stream_usage=True,
    temperature=0.0,
    max_tokens=2000,
    # timeout=None,
    #reasoning_effort="low",
    # max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instead of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [ ]:
# Install required packages
# pip install langgraph langchain langchain-openai

from typing import TypedDict, List, Annotated
import operator
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

# ============================
# 1. Define the Agent State
# ============================
class AgentState(TypedDict):
    """State for our DeepAgent"""
    messages: Annotated[List[str], operator.add]  # Conversation history
    current_step: str                              # Current execution step
    result: str                                    # Final result
    max_iterations: int                            # Safety limit
    iteration_count: int                           # Track iterations

# ============================
# 2. Define Node Functions
# ============================
def planner_node(state: AgentState) -> AgentState:
    """Planner: Decides what to do next"""
    print(f"🧠 Planner Step {state['iteration_count'] + 1}")
    
    # In a real implementation, this would use an LLM
    # For this example, we'll use simple logic
    messages = state['messages']
    last_message = messages[-1] if messages else "Start task"
    
    if "calculate" in last_message.lower():
        state['current_step'] = "calculator"
    elif "search" in last_message.lower():
        state['current_step'] = "search"
    elif "summarize" in last_message.lower():
        state['current_step'] = "summarizer"
    else:
        state['current_step'] = "responder"
    
    return state

def calculator_node(state: AgentState) -> AgentState:
    """Calculator: Performs calculations"""
    print("🧮 Calculator executing...")
    last_message = state['messages'][-1]
    
    # Simple calculation extraction
    if "2+2" in last_message:
        state['result'] = "The result of 2+2 is 4"
    elif "10*5" in last_message:
        state['result'] = "The result of 10*5 is 50"
    else:
        state['result'] = "I can perform simple calculations"
    
    return state

def search_node(state: AgentState) -> AgentState:
    """Search: Simulates information retrieval"""
    print("🔍 Search executing...")
    query = state['messages'][-1]
    state['result'] = f"Search results for '{query}': [Result 1, Result 2, Result 3]"
    return state

def summarizer_node(state: AgentState) -> AgentState:
    """Summarizer: Creates summaries"""
    print("📝 Summarizer executing...")
    text = state['messages'][-1]
    # Simple summary logic
    words = text.split()
    summary = ' '.join(words[:10]) + "..." if len(words) > 10 else text
    state['result'] = f"Summary: {summary}"
    return state

def responder_node(state: AgentState) -> AgentState:
    """Responder: Generates final response"""
    print("💬 Responder executing...")
    
    if state.get('result'):
        response = state['result']
    else:
        response = f"I've completed step: {state['current_step']}"
    
    state['messages'].append(f"Agent: {response}")
    return state

def should_continue(state: AgentState) -> str:
    """Router: Decides whether to continue or end"""
    state['iteration_count'] += 1
    
    # Check if we have a result or hit max iterations
    if state.get('result') or state['iteration_count'] >= state['max_iterations']:
        return "end"
    return "continue"

# ============================
# 3. Build the LangGraph
# ============================
def build_deep_agent_graph():
    """Builds the complete agent workflow graph"""
    
    # Create workflow builder
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("planner", planner_node)
    workflow.add_node("calculator", calculator_node)
    workflow.add_node("search", search_node)
    workflow.add_node("summarizer", summarizer_node)
    workflow.add_node("responder", responder_node)
    
    # Set entry point
    workflow.set_entry_point("planner")
    
    # Define conditional edges from planner
    workflow.add_conditional_edges(
        "planner",
        lambda state: state['current_step'],
        {
            "calculator": "calculator",
            "search": "search", 
            "summarizer": "summarizer",
            "responder": "responder"
        }
    )
    
    # Add edges from action nodes to responder
    workflow.add_edge("calculator", "responder")
    workflow.add_edge("search", "responder")
    workflow.add_edge("summarizer", "responder")
    
    # Add conditional edge from responder
    workflow.add_conditional_edges(
        "responder",
        should_continue,
        {
            "continue": "planner",
            "end": END
        }
    )
    
    # Add memory for state persistence
    memory = MemorySaver()
    
    # Compile the graph
    app = workflow.compile(checkpointer=memory)
    
    return app

# ============================
# 4. Run the DeepAgent
# ============================
def run_deep_agent():
    """Execute the DeepAgent with different tasks"""
    
    # Build the agent graph
    agent = build_deep_agent_graph()
    
    # Test cases
    test_tasks = [
        "Calculate 2+2 for me",
        "Search for information about AI",
        "Summarize this long text about machine learning models"
    ]
    
    for task in test_tasks:
        print(f"\n{'='*50}")
        print(f"Task: {task}")
        print('='*50)
        
        # Initial state
        initial_state = {
            "messages": [f"User: {task}"],
            "current_step": "",
            "result": "",
            "max_iterations": 5,
            "iteration_count": 0
        }
        
        # Execute the agent
        config = {"configurable": {"thread_id": "1"}}
        final_state = agent.invoke(initial_state, config)
        
        # Display results
        print(f"\n📊 Final Result: {final_state.get('result', 'No result')}")
        print(f"📈 Total Steps: {final_state['iteration_count']}")
        print(f"💭 Conversation History:")
        for msg in final_state['messages']:
            print(f"  - {msg}")

# ============================
# 5. Advanced: LLM-Powered Version
# ============================
class LLMDeepAgent:
    """More advanced version using actual LLMs"""
    
    def __init__(self, model_name="gpt-3.5-turbo"):
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        self.graph = self._build_llm_graph()
    
    def _build_llm_graph(self):
        """Build graph with LLM-based decision making"""
        
        def llm_planner(state):
            """Use LLM to decide next step"""
            prompt = f"""
            Based on the conversation, decide the next action:
            Conversation: {state['messages']}
            
            Choose from: [calculator, search, summarizer, responder, end]
            
            Return only the action name.
            """
            
            response = self.llm.invoke(prompt)
            state['current_step'] = response.content.strip().lower()
            return state
        
        # Similar graph structure but with LLM planner
        workflow = StateGraph(AgentState)
        workflow.add_node("planner", llm_planner)
        # ... add other nodes as before
        
        return workflow.compile()

# ============================
# 6. Main Execution
# ============================
if __name__ == "__main__":
    print("🚀 LangGraph DeepAgent Demonstration")
    print("="*50)
    
    # Run the basic agent
    run_deep_agent()
    
    print("\n\n🔧 Key Concepts Demonstrated:")
    print("1. State Management: TypedDict for agent state")
    print("2. Node Functions: Specialized components")
    print("3. Conditional Routing: Dynamic workflow paths")
    print("4. Memory/Checkpoints: State persistence")
    print("5. Iterative Processing: Multiple reasoning steps")